In [1]:
import numpy as np
import pandas as pd
from math import radians
from scipy.optimize import least_squares
import sys

def model_xy(t,theta_rad,M,X):
    a = np.exp(M*np.abs(t))*np.sin(0.3*t)
    x = t*np.cos(theta_rad) - a*np.sin(theta_rad) + X
    y = 42 + t*np.sin(theta_rad) + a*np.cos(theta_rad)
    return x, y

def residuals_known_t(params,t,x_obs,y_obs):
    theta_deg, M, X = params
    theta_rad = radians(theta_deg)
    x_model,y_model = model_xy(t,theta_rad,M,X)
    res = np.concatenate([(x_model-x_obs).ravel(),(y_model-y_obs).ravel()])
    return res

def residuals_unknown_t(full_params,x_obs,y_obs):
    theta_deg = full_params[0]
    M = full_params[1]
    X = full_params[2]
    t = full_params[3:]
    theta_rad = radians(theta_deg)
    x_model,y_model = model_xy(t,theta_rad,M,X)
    res = np.concatenate([(x_model-x_obs).ravel(),(y_model-y_obs).ravel()])
    return res

def run():
    try:
        df = pd.read_csv('xy_data.csv')
    except Exception as e:
        print('Error reading xy_data.csv:',e)
        sys.exit(1)
    cols = [c.lower().strip() for c in df.columns]
    if 't' in cols and ('x' in cols and 'y' in cols or 'x_obs' in cols and 'y_obs' in cols):
        t = df.iloc[:,cols.index('t')].values.astype(float)
        x_col = 'x' if 'x' in cols else 'x_obs'
        y_col = 'y' if 'y' in cols else 'y_obs'
        x_obs = df[x_col].values.astype(float)
        y_obs = df[y_col].values.astype(float)
        p0 = np.array([25.0,0.0,10.0])
        lower = [0.000001,-0.05,0.0]
        upper = [50.0,0.05,100.0]
        res = least_squares(residuals_known_t,p0,args=(t,x_obs,y_obs),bounds=(lower,upper),ftol=1e-12,xtol=1e-12,max_nfev=20000)
        theta_deg,M,X = res.x
        theta_rad = radians(theta_deg)
        x_model,y_model = model_xy(t,theta_rad,M,X)
        L1 = np.sum(np.abs(x_model-x_obs)+np.abs(y_model-y_obs))
        print('theta_deg',theta_deg)
        print('M',M)
        print('X',X)
        print('L1_total',L1)
    else:
        if df.shape[1]==2:
            x_obs = df.iloc[:,0].values.astype(float)
            y_obs = df.iloc[:,1].values.astype(float)
        elif df.shape[1]==3:
            t = df.iloc[:,0].values.astype(float)
            x_obs = df.iloc[:,1].values.astype(float)
            y_obs = df.iloc[:,2].values.astype(float)
        else:
            print('Unrecognized CSV format. Expected columns (t,x,y) or (x,y) or three columns (x,y,t).')
            sys.exit(1)
        n = x_obs.size
        theta0 = 25.0
        M0 = 0.0
        X0 = 10.0
        t0 = np.linspace(6.0,60.0,n)
        full0 = np.concatenate(([theta0,M0,X0],t0))
        lower = np.concatenate(([0.000001,-0.05,0.0],np.full(n,6.0)))
        upper = np.concatenate(([50.0,0.05,100.0],np.full(n,60.0)))
        res = least_squares(lambda p: residuals_unknown_t(p,x_obs,y_obs),full0,bounds=(lower,upper),ftol=1e-12,xtol=1e-12,max_nfev=40000)
        theta_deg = res.x[0]
        M = res.x[1]
        X = res.x[2]
        t_fit = res.x[3:]
        theta_rad = radians(theta_deg)
        x_model,y_model = model_xy(t_fit,theta_rad,M,X)
        L1 = np.sum(np.abs(x_model-x_obs)+np.abs(y_model-y_obs))
        print('theta_deg',theta_deg)
        print('M',M)
        print('X',X)
        print('L1_total',L1)
        out = pd.DataFrame({'t_fit':t_fit,'x_obs':x_obs,'y_obs':y_obs,'x_model':x_model,'y_model':y_model})
        out.to_csv('fit_with_estimated_t.csv',index=False)
        print('Saved fit_with_estimated_t.csv with per-point t estimates')

if __name__=='__main__':
    run()


theta_deg 29.99997300498904
M 0.029999997169778803
X 54.999998352630804
L1_total 0.003984133333872819
Saved fit_with_estimated_t.csv with per-point t estimates
